In [2]:
pip install statsmodels

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
You should consider upgrading via the '/opt/conda/envs/python35-paddle120-env/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
#载入类库
import warnings
import pandas as pd
import numpy as np
import lightgbm as lgb
from matplotlib import pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.ar_model import AR
from sklearn.model_selection import KFold,train_test_split
from sklearn.metrics import mean_squared_error
warnings.filterwarnings('ignore')

In [3]:
#读取数据
df1=pd.read_csv('/home/aistudio/data/data143438/area_passenger_index.csv',header=None
                    ,names=['ID','time','people_index'])
df2=pd.read_csv('/home/aistudio/data/data143438/area_passenger_info.csv',header=None
                    ,names=['ID','area_name','area_type','Center_x','Center_y','Grid_x','Grid_y','area'])
df3=pd.read_csv('/home/aistudio/data/data143438/migration_index.csv',header=None
                ,names=['date','departure_province','departure_city','arrival_province','arrival_city','index'])
df4=pd.read_csv('/home/aistudio/data/data143438/grid_strength.csv',header=None
                    ,names=['hour','start_grid_x','start_grid_y','end_grid_x','end_grid_y','Index'])
sub=pd.read_csv('/home/aistudio/data/data143438/test_submit_example.csv',header=None
                    ,names=['ID','time','people_index'])

In [4]:
#简要查看数据
df2.head()

,ID,area_name,area_type,Center_x,Center_y,Grid_x,Grid_y,area
0,1,北坞公园,旅游景点;公园,116.256586,39.986913,116.256713,39.987525,395925.609375
1,2,古塔公园,旅游景点;公园,116.535404,39.878676,116.535232,39.878140,575268.839844
2,3,中央戏剧学院,教育培训;高等院校,116.404667,39.935999,116.404148,39.936165,17024.636719
3,4,丰联广场,购物;购物中心,116.438556,39.923713,116.438117,39.924203,11695.019531
4,5,中华女子学院,教育培训;高等院校,116.435352,39.994935,116.434508,39.994636,83053.546875


In [5]:
#对df1时间格式处理
df1['time']=pd.to_datetime(df1['time'],format='%Y%m%d%H')
df1['day']=df1['time'].dt.day.apply(lambda x:x-17 if x>=17 else x+14)
df1['weekday']=df1['time'].dt.weekday
df1['hour']=df1['time'].dt.hour

In [6]:
#按时间、地点取平均    
dict_wd=dict(df1.groupby(['weekday']).mean()['people_index'])
dict_h=dict(df1.groupby(['hour']).mean()['people_index'])
dict_id=dict(df1.groupby(['ID']).mean()['people_index'])

In [7]:
#添加处理后特征
df1['weekday_encoded']=df1['weekday'].map(dict_wd)
df1['hour_encoded']=df1['hour'].map(dict_h)
df1['ID_encoded']=df1['ID'].map(dict_id)

In [8]:
#对运行后展示进行无界处理
pd.set_option('display.width',None)

In [9]:
#数据备份处理
train_data=df1.copy()

In [10]:
#合并数据集并开始处理df2
train_data=pd.merge(df1,df2,on=['ID'])

In [11]:
#按区域类别分类取平均
dict_at=dict(train_data.groupby(['area_type']).mean()['people_index'])

In [12]:
 #添加处理后特征
train_data['area_type_encoded']=train_data['area_type'].map(dict_at)

In [13]:
#去除较为无关的特征
train_data.drop(['area_name','area_type'],axis=1,inplace=True)

In [14]:
#对df3时间格式处理
df3['date']=pd.to_datetime(df3['date'],format='%Y%m%d')
df3['day_encode']=df3['date'].dt.day.apply(lambda x:x-17 if x>=17 else x+14)


In [15]:

#对迁移数据进行处理
df3['city']=df3['arrival_city'].apply(lambda x:1 if x=='北京市' else 0)
inbj=df3[df3['city']==1]
outbj=df3[df3['city']==0]
index_mean_in=inbj.groupby(['date']).mean()['index']
index_std_in=inbj.groupby(['date']).std()['index']
index_mean_in_copy=index_mean_in.copy()
index_mean_in_copy.index=np.arange(30)
dict4=dict(index_mean_in_copy)
index_std_in_copy=index_std_in.copy()
index_std_in_copy.index=np.arange(30)
dict5=dict(index_std_in_copy)
index_mean_out=outbj.groupby(['date']).mean()['index']
index_std_out=outbj.groupby(['date']).std()['index']
index_mean_out_copy=index_mean_out.copy()
index_mean_out_copy.index=np.arange(30)
dict6=dict(index_mean_out_copy)
index_std_out_copy=index_std_out.copy()
index_std_out_copy.index=np.arange(30)
dict7=dict(index_std_out_copy)


train_data['index_mean_in']=train_data['day'].map(dict4)
train_data['index_std_in']=train_data['day'].map(dict5)
train_data['index_mean_out']=train_data['day'].map(dict6)
train_data['index_std_out']=train_data['day'].map(dict7)

In [16]:
#用ar模型进行时间序列预测
ar_model1=AR(index_mean_in).fit()
pred_index_mean_in=ar_model1.predict('20200216','20200224',dynamic=True)
ar_model3=AR(index_mean_out).fit()
pred_index_mean_out=ar_model3.predict('20200216','20200224',dynamic=True)
ar_model2=AR(index_std_in).fit()
pred_index_std_in=ar_model2.predict('20200216','20200224',dynamic=True)
ar_model4=AR(index_std_out).fit()
pred_index_std_out=ar_model4.predict('20200216','20200224',dynamic=True)


In [ ]:
#对df4进行数据处理
dict_hi=dict(df4.groupby(['hour']).mean()['Index'])
train_data['hi_encoded']=train_data['hour'].map(dict_hi)

In [17]:
#对测试数据的时间进行处理以便构建模型
test_data=sub.copy()
test_data.drop(['people_index'],axis=1,inplace=True)
test_data['time']=pd.to_datetime(test_data['time'],format='%Y%m%d%H')
test_data['weekday']=test_data['time'].dt.weekday
test_data['hour']=test_data['time'].dt.hour
test_data['weekday_encoded']=test_data['weekday'].map(dict_wd)
test_data['hour_encoded']=test_data['hour'].map(dict_h)
test_data['ID_encoded']=test_data['ID'].map(dict_id)
test_data['day']=test_data['time'].dt.day.apply(lambda x:x+14)


In [18]:
pred_index_mean_in.index=np.arange(30,39,1)
dict_pim=dict(pred_index_mean_in)
pred_index_std_in.index=np.arange(30,39,1)
dict_pis=dict(pred_index_std_in)
pred_index_mean_out.index=np.arange(30,39,1)
dict_pim1=dict(pred_index_mean_out)
pred_index_std_out.index=np.arange(30,39,1)
dict_pis1=dict(pred_index_std_out)
test_data['index_mean_in']=test_data['day'].map(dict_pim)
test_data['index_std_in']=test_data['day'].map(dict_pis)
test_data['index_mean_out']=test_data['day'].map(dict_pim1)
test_data['index_std_out']=test_data['day'].map(dict_pis1)

test_data=pd.merge(test_data,df2,on=['ID'])
test_data.drop(['Center_x','Center_y','Grid_x','Grid_y'],axis=1,inplace=True)
test_data['hi_encoded']=test_data['hour'].map(dict_hi)
test_data['area_type_encoded']=test_data['area_type'].map(dict_at)
test_data.drop(['ID','time','weekday','hour','day','area_name','area_type'],axis=1,inplace=True)
train_data=train_data[train_data['day']>=16]
train_data.index=np.arange(train_data.shape[0])
target=train_data['people_index']

In [19]:
train_data.drop(['people_index','day','ID','weekday','hour','Center_x','Center_y','Grid_x','Grid_y','time']
                    ,axis=1,inplace=True)
print('train_data.head()\n',train_data.head())
test_new=pd.DataFrame([test_data['weekday_encoded']#周几平均
                         ,test_data['hour_encoded']#小时平均
                         ,test_data['ID_encoded']#区域平均
                         ,test_data['area']#区域面积
                         ,test_data['area_type_encoded']#区域类型平均
                         ,test_data['index_mean_in']#迁入平均
                         ,test_data['index_std_in']#迁入方差
                         ,test_data['index_mean_out']#迁出平均
                         ,test_data['index_std_out']#迁出方差
                         ,test_data['hi_encoded']]).T#网格间流动小时平均
print('test_new.head()\n',test_new.head())

train_data.head()
    weekday_encoded  hour_encoded  ID_encoded           area  \
0        27.552537     12.499074    4.835556  395925.609375   
1        27.552537      8.617108    4.835556  395925.609375   
2        27.552537      7.245714    4.835556  395925.609375   
3        27.552537      6.373882    4.835556  395925.609375   
4        27.552537      6.205978    4.835556  395925.609375   

   area_type_encoded  index_mean_in  index_std_in  index_mean_out  \
0          10.071021       0.016892      0.042052        0.003928   
1          10.071021       0.016892      0.042052        0.003928   
2          10.071021       0.016892      0.042052        0.003928   
3          10.071021       0.016892      0.042052        0.003928   
4          10.071021       0.016892      0.042052        0.003928   

   index_std_out  hi_encoded  
0       0.015576    0.218439  
1       0.015576    0.219937  
2       0.015576    0.202704  
3       0.015576    0.194659  
4       0.015576    0.264326  
t

In [ ]:
    import  lightgbm as lgb
    model_lgb=lgb.LGBMRegressor(num_leaves=20
                                ,max_depth=6
                                ,learning_rate=0.09
                                ,n_estimators=1100
                                ,n_jobs=-1
                                )#所有core进行计算
    x_train,x_test,y_train,y_test=train_test_split(train_data,target,test_size=0.2)
    model_lgb.fit(x_train,y_train)
    y_hat=model_lgb.predict(x_test)
    y_hat[y_hat<0]=0
    print(1/(mean_squared_error(y_test,y_hat)**0.5+1))
    keys=[]
    sk=KFold(n_splits=10,shuffle=True)
    for train,test in sk.split(train_data,target):
        x_train=train_data.iloc[train]
        y_train=target.iloc[train]
        x_test=train_data.iloc[test]
        y_test=target.iloc[test]
        model_lgb.fit(x_train,y_train)
        y_hat=model_lgb.predict(x_test)
        y_hat[y_hat<0]=0
        print(1/(mean_squared_error(y_test,y_hat)**0.5+1))
        keys.append(model_lgb.predict(test_new))
    keys=np.array(keys)
    km=keys.mean(axis=0)
    km[km<0]=0
    sub.iloc[:,2]=km
    sub.to_csv('result.csv',encoding='utf-8',header=None,index=None)

In [4]:

result1=pd.read_csv('result.csv',header=None
                    ,names=['ID','time','people_index'])
result2=pd.read_csv('result1.csv',header=None
                    ,names=['ID','time','people_index'])
result1['people_index'] = result1.apply(lambda x: (x['people_index'] + result2['people_index'])/2, axis=1)
result1.head()
# result1.to_csv('result_mix.csv',encoding='utf-8',header=None,index=None)

KeyboardInterrupt: 